In [ ]:

import tensorflow as tf
import numpy as np
import os
import time

In [ ]:
# Download and prepare the dataset
path_to_file = tf.keras.utils.get_file(
    'shakespeare.txt', 
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
)
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [4]:
# Print length and preview
print(f'Length of text: {len(text)} characters')
print(f'First 250 characters:\n{text[:250]}')

Length of text: 1115394 characters
First 250 characters:
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [ ]:
# Process the text
# Create vocabulary of unique characters
vocab = sorted(set(text))
print(f'Total unique characters: {len(vocab)}')

Total unique characters: 65


In [6]:
# Create mapping from character to index and vice versa
char2index = {char: idx for idx, char in enumerate(vocab)}
index2char = {idx: char for idx, char in enumerate(vocab)}


In [7]:
# Convert text to numerical representation
text_as_int = np.array([char2index[c] for c in text])
print(f'Text mapped to integers: {text_as_int[:20]}')

Text mapped to integers: [18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56]


In [ ]:
# Create training examples and targets
seq_length = 100  # Length of sequence for a training example
examples_per_epoch = len(text) // (seq_length + 1)

In [9]:
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

I0000 00:00:1742749198.671887   31063 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9682 MB memory:  -> device: 0, name: NVIDIA RTX 3500 Ada Generation Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [ ]:
# Create training batches
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

print(f"Dataset shape: {dataset}")

Dataset shape: <_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>


In [ ]:
# Build the model
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1536

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        # Add an InputLayer first to handle the batch size
        tf.keras.layers.InputLayer(input_shape=(None,), batch_size=batch_size),
        
        # Embedding layer
        tf.keras.layers.Embedding(
            input_dim=vocab_size,
            output_dim=embedding_dim
        ),
        
        # LSTM layer
        tf.keras.layers.LSTM(
            units=rnn_units,
            return_sequences=True,
            stateful=True,
            recurrent_initializer=tf.keras.initializers.GlorotNormal()
        ),
        
        # Dense output layer
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [12]:
model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)
model.summary()

/home/kacper/Y4S2/DataScience/DataScienceSem2/.venv/lib64/python3.12/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (64, None, 256)        │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (64, None, 1536)       │    11,016,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (64, None, 65)         │        99,905 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,132,737 (42.47 MB)

 Trainable params: 11,132,737 (42.47 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Define loss function and compile
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Train the model
EPOCHS = 40

# Create a directory to save checkpoints
checkpoint_dir = './training_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

# Add .weights.h5 extension as required by Keras
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

# Train the model
history = model.fit(
    dataset,
    epochs=EPOCHS,
    callbacks=[checkpoint_callback]
)

Epoch 1/40


  1/172 ━━━━━━━━━━━━━━━━━━━━ 4:55 2s/step - loss: 4.1734

I0000 00:00:1742749200.854320   31336 cuda_dnn.cc:529] Loaded cuDNN version 90300


172/172 ━━━━━━━━━━━━━━━━━━━━ 7s 31ms/step - loss: 2.9715
Epoch 2/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - loss: 1.9046
Epoch 3/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - loss: 1.6218
Epoch 4/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - loss: 1.4831
Epoch 5/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - loss: 1.3978
Epoch 6/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - loss: 1.3375
Epoch 7/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - loss: 1.2902
Epoch 8/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - loss: 1.2499
Epoch 9/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - loss: 1.2065
Epoch 10/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - loss: 1.1655
Epoch 11/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - loss: 1.1227
Epoch 12/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - loss: 1.0761
Epoch 13/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - loss: 1.0291
Epoch 14/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - loss: 0.9756
Epoch 15/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step 

In [ ]:
def generate_text(model, start_string, num_generate=1000, temperature=1.0):
    # Converting start string to numbers
    input_indices = [char2index[c] for c in start_string]
    
    # Create a batch of identical inputs to match BATCH_SIZE
    input_indices = tf.expand_dims(input_indices, 0)
    input_indices = tf.repeat(input_indices, BATCH_SIZE, axis=0)
    
    # Empty string to store result
    text_generated = []
    
    # Reset states manually by accessing the LSTM layer directly
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.LSTM):
            layer.reset_states()
    
    for i in range(num_generate):
        predictions = model(input_indices)
        # Only use first sequence in batch for prediction
        predictions = predictions[0]
        
        # Use temperature for sampling
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(tf.expand_dims(predictions[-1], 0), 
                                           num_samples=1)[-1, 0].numpy()
        
        # Add predicted character
        text_generated.append(index2char[predicted_id])
        
        # Update input with new character for all sequences in batch
        next_char = tf.expand_dims([predicted_id], 0)
        next_char = tf.repeat(next_char, BATCH_SIZE, axis=0)
        
        # Shift input and add new character
        if input_indices.shape[1] > 1:
            input_indices = tf.concat([input_indices[:, 1:], next_char], axis=1)
        else:
            input_indices = next_char
    
    return start_string + ''.join(text_generated)

# Generate text examples
print("\nGenerating Shakespeare text...")
print("\nRomeo's lines:")
print(generate_text(model, start_string="ROMEO: My love for Juliet is ", temperature=1.0))





Generating Shakespeare text...

Romeo's lines:
ROMEO: My love for Juliet is befal.

HENRY BOLINGBROKE:
More hold that I said so? lay and beggary
Then resolute as we are.

GRUMIO:
My cake is dough; but I'll in among these words,
Infer favours shall be fearful.

HENRY BOLINGBROKE:
First, heaven be the worst that I should speak,
Before I stay: but I have stay'd
To tin thou shalt content that thou belike.'

Second Citizen:
Think you me all this whileld?

Shepherd:
Take hands, a bargain! and so doth he; and then
Lady:
Madam, I have one, and therein
Of March wash y sweet saluteth me?
Young son, it argues a thousand grains
And her re usurp'd or fruit;
Which was an adulteress;
For to my rest that love what he would have strickenge-stand, the prince Frorth,
Whose strange still, which will inform thee here.

ROMEO:
I have night's cloak:
Who now the priest shout self-born in all:
In all the throne names and undertakes,
And I will take thy word: yet if thou swear'st,
A thousand thanks, breed thee

In [22]:
print("\nHamlet's soliloquy: ")
print(generate_text(model, start_string="HAMLET: To be, or not to be", temperature=0.4))


Hamlet's soliloquy: 
HAMLET: To be, or not to be part,
Let me except the time to come.

JULIET:
O God, I have a noble memory; what of him?
Our slaught and grief is now born to repend.

JULIET:
I do confess it, Tranio, for the face
Be merry, gods. Imply that know it.

GLOUCESTER:
The queen is valued thirty.

WARWICK:
How now, my heart! how now, what news with you,
She hath a poor men of this.

JULIET:
I will tell thee, Let me be thus bold.

LADY ANNE:
I would I knew thy heart.

GLOUCESTER:

CLARENCE:

KING EDWARD IV:
Nay, this shall be.

FRIAR LAURENCE:
So smile the heavens and them
At hand that I may be: but I cannot
Believe this crack to be ingrateful,
were to draw in many a tear
And stop the rising of blood-sucking but the base court of the
two of the house of the Montagues.

SAMPSON:
My noble lords and cousin Buckingham,
Then cursed she Richard. O, remember, God
To hear her prayers for the fair Bianca:
And thou shalt tell the truth.

BRUTUS:
Being the red plague rid you
For learnin

In [20]:
print("\nCreative writing:")
print(generate_text(model, start_string="Once upon a time", temperature=1.5))


Creative writing:
Once upon a time monur, while mef we entreat him.

PETRUCHIO:
Fhy, Harry, we sings down toh,
Put to the friar,
Yet,--ay, er:'
Those old deserved it;
And therein fears; the bay-lenced
Than Bience did for, i'er hide,
Which with suspicion!

DUKE VINCENTIO:
He proseeveth g, ar I dare swear thio! how he's unto revenge,
Which ICHARD:
Then oncl we speak, of all
The charges, of the third a
diber hoarser'd: I will tell you 'fore.'

WARWIO:
My Lord!

Ghost of Hereford,?

HENRY BOLINGBROKE:
Noth, and yoursband,
You will descends.

LUCENTIO:
Come, come, deny along
Of your queen as yours but use her horn-dids death forsaid
unto the ear, when
But yet nt.

QUEEN ELIZABETH:
Thy life resolved.

Shepherd:
Ay, school-mages, kepe all avishs, yet look where they quarter'd broke a jadnes.

MERCUTIO:
Confusion!

LUCIO:
Down, down toRMASER:
Beshall yield you, ele?

Page:
His name, my lord; and
love tosh, entertain use tongue in fair proconsA:
Not over of these are my ships and great Italy?
